# **Yelp API Core**

_John Andrew Dixon_

---

##### **Imports**

In [10]:
import json
import pandas as pd
from yelpapi import YelpAPI

---

## **Load & Connect API Key**

In [11]:
with open('/Users/johna/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [12]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

## **Simple API Usage**

In [18]:
# First page query
results = yelp_api.search_query(location='Riverside, CA',
                                term='Beer')
results['total']

2200

In [19]:
beer0 = pd.DataFrame(results["businesses"])
beer0.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,4_v8dpfRWujLOHhz1QChww,packinghouse-brewing-company-riverside,Packinghouse Brewing Company,https://s3-media1.fl.yelpcdn.com/bphoto/gKWLKd...,False,https://www.yelp.com/biz/packinghouse-brewing-...,161,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 33.95426, 'longitude': -117.43978}",[],$$,"{'address1': '6421 Central Ave', 'address2': '...",+19513339261,(951) 333-9261,3935.257528
1,bMUWADej_vl2oaSY1kYTSg,the-brickwood-riverside-2,The Brickwood,https://s3-media1.fl.yelpcdn.com/bphoto/VSsnD0...,False,https://www.yelp.com/biz/the-brickwood-riversi...,194,"[{'alias': 'beerbar', 'title': 'Beer Bar'}, {'...",4.5,"{'latitude': 33.9835327571997, 'longitude': -1...",[],$$,"{'address1': '3653 Main St', 'address2': '', '...",+19512502712,(951) 250-2712,4817.051192
2,IyvTC0XbOd0szRd7GXt6EA,thompson-brewing-riverside-2,Thompson Brewing,https://s3-media4.fl.yelpcdn.com/bphoto/-YWINu...,False,https://www.yelp.com/biz/thompson-brewing-rive...,444,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 33.909111, 'longitude': -117.449463}","[delivery, pickup]",$$,"{'address1': '9900 Indiana Ave', 'address2': '...",+19512265414,(951) 226-5414,6207.518133
3,2AOHbEGe5gQAM3GRtdXfuQ,beerfarm-riverside,Beerfarm,https://s3-media3.fl.yelpcdn.com/bphoto/mFZAAk...,False,https://www.yelp.com/biz/beerfarm-riverside?ad...,111,"[{'alias': 'beerbar', 'title': 'Beer Bar'}]",4.5,"{'latitude': 33.984484, 'longitude': -117.374656}",[delivery],$$,"{'address1': '3605 Market St', 'address2': '',...",,,4865.157550
4,foVIVuwSAG1UymdnizT8NA,route-30-brewing-company-riverside,Route 30 Brewing Company,https://s3-media2.fl.yelpcdn.com/bphoto/Rg3CuZ...,False,https://www.yelp.com/biz/route-30-brewing-comp...,113,"[{'alias': 'breweries', 'title': 'Breweries'}]",5.0,"{'latitude': 33.90982569681414, 'longitude': -...",[],$,"{'address1': '9860 Indiana Ave', 'address2': '...",+19517767083,(951) 776-7083,5992.610879


In [20]:
# Quick Test Query
results = yelp_api.search_query(location='Riverside, CA',
                                term='Beer',
                                offset=20)
results['total']

2200

In [21]:
beer1 = pd.DataFrame(results["businesses"])
beer1.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,tO8LLGcR38SLa1fPZI8how,lake-arrowhead-brewing-lake-arrowhead,Lake Arrowhead Brewing,https://s3-media3.fl.yelpcdn.com/bphoto/CiMkwI...,False,https://www.yelp.com/biz/lake-arrowhead-brewin...,114,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 34.249977, 'longitude': -117.1937...",[],$$,"{'address1': '329 Villa Way', 'address2': '', ...",,,38756.466301
1,KTQSNF3XH7sx81xe7gwTVw,hangar-24-craft-brewing-redlands,Hangar 24 Craft Brewing,https://s3-media2.fl.yelpcdn.com/bphoto/WOo0NL...,False,https://www.yelp.com/biz/hangar-24-craft-brewi...,703,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.0,"{'latitude': 34.083151, 'longitude': -117.141796}",[],$$,"{'address1': '1710 Sessums Dr', 'address2': ''...",+19092428111,(909) 242-8111,28183.661910
2,TRI-OJfrvyZ74rtqlFSFxw,heroes-restaurant-and-brewery-riverside,Heroes Restaurant and Brewery,https://s3-media4.fl.yelpcdn.com/bphoto/PIDn0K...,False,https://www.yelp.com/biz/heroes-restaurant-and...,2238,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 33.9814567544957, 'longitude': -1...",[delivery],$$,"{'address1': '3397 Mission Inn Ave', 'address2...",+19512480722,(951) 248-0722,4796.739615
3,1nmWQsZlJZo4OpJZ360BZg,greywolf-brewing-norco-3,GreyWolf Brewing,https://s3-media2.fl.yelpcdn.com/bphoto/iOoidU...,False,https://www.yelp.com/biz/greywolf-brewing-norc...,61,"[{'alias': 'brewpubs', 'title': 'Brewpubs'}]",5.0,"{'latitude': 33.9279090300886, 'longitude': -1...",[],$$,"{'address1': '1780 Town And Country Dr', 'addr...",+19514795512,(951) 479-5512,14968.937324
4,pacFk-csrp5LrrFIQTpnyw,brew-haven-ontario,Brew Haven,https://s3-media2.fl.yelpcdn.com/bphoto/_CQzrp...,False,https://www.yelp.com/biz/brew-haven-ontario?ad...,103,"[{'alias': 'beerbar', 'title': 'Beer Bar'}]",4.5,"{'latitude': 34.0001988, 'longitude': -117.578...",[],$$,"{'address1': '3430 E Ontario Ranch Rd', 'addre...",,,17660.758807


In [24]:
brews = pd.concat([beer0, beer1], ignore_index=True)
display(brews.head(), brews.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,4_v8dpfRWujLOHhz1QChww,packinghouse-brewing-company-riverside,Packinghouse Brewing Company,https://s3-media1.fl.yelpcdn.com/bphoto/gKWLKd...,False,https://www.yelp.com/biz/packinghouse-brewing-...,161,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 33.95426, 'longitude': -117.43978}",[],$$,"{'address1': '6421 Central Ave', 'address2': '...",+19513339261,(951) 333-9261,3935.257528
1,bMUWADej_vl2oaSY1kYTSg,the-brickwood-riverside-2,The Brickwood,https://s3-media1.fl.yelpcdn.com/bphoto/VSsnD0...,False,https://www.yelp.com/biz/the-brickwood-riversi...,194,"[{'alias': 'beerbar', 'title': 'Beer Bar'}, {'...",4.5,"{'latitude': 33.9835327571997, 'longitude': -1...",[],$$,"{'address1': '3653 Main St', 'address2': '', '...",+19512502712,(951) 250-2712,4817.051192
2,IyvTC0XbOd0szRd7GXt6EA,thompson-brewing-riverside-2,Thompson Brewing,https://s3-media4.fl.yelpcdn.com/bphoto/-YWINu...,False,https://www.yelp.com/biz/thompson-brewing-rive...,444,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 33.909111, 'longitude': -117.449463}","[delivery, pickup]",$$,"{'address1': '9900 Indiana Ave', 'address2': '...",+19512265414,(951) 226-5414,6207.518133
3,2AOHbEGe5gQAM3GRtdXfuQ,beerfarm-riverside,Beerfarm,https://s3-media3.fl.yelpcdn.com/bphoto/mFZAAk...,False,https://www.yelp.com/biz/beerfarm-riverside?ad...,111,"[{'alias': 'beerbar', 'title': 'Beer Bar'}]",4.5,"{'latitude': 33.984484, 'longitude': -117.374656}",[delivery],$$,"{'address1': '3605 Market St', 'address2': '',...",,,4865.157550
4,foVIVuwSAG1UymdnizT8NA,route-30-brewing-company-riverside,Route 30 Brewing Company,https://s3-media2.fl.yelpcdn.com/bphoto/Rg3CuZ...,False,https://www.yelp.com/biz/route-30-brewing-comp...,113,"[{'alias': 'breweries', 'title': 'Breweries'}]",5.0,"{'latitude': 33.90982569681414, 'longitude': -...",[],$,"{'address1': '9860 Indiana Ave', 'address2': '...",+19517767083,(951) 776-7083,5992.610879


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
35,efPinYIq7HdztJZt1gJutQ,fire-up-bar-and-grill-riverside,Fire Up Bar & Grill,https://s3-media1.fl.yelpcdn.com/bphoto/PRDiej...,False,https://www.yelp.com/biz/fire-up-bar-and-grill...,482,"[{'alias': 'bars', 'title': 'Bars'}]",4.0,"{'latitude': 33.9814821604937, 'longitude': -1...",[delivery],$$,"{'address1': '3750 University Ave', 'address2'...",+19512899071,(951) 289-9071,4552.255351
36,DWih10sVFsQVhxwBp_rIGA,more-beer-more-wine-riverside,"More Beer, More Wine",https://s3-media3.fl.yelpcdn.com/bphoto/izTArn...,False,https://www.yelp.com/biz/more-beer-more-wine-r...,58,"[{'alias': 'brewingsupplies', 'title': 'Brewin...",4.5,"{'latitude': 34.00109, 'longitude': -117.3442}",[],$$,"{'address1': '1506 Columbia Ave', 'address2': ...",+19517799971,(951) 779-9971,7903.070563
37,eMYsC_gh4nXM3IKtWMXsXQ,dog-haus-eastvale-2,Dog Haus,https://s3-media2.fl.yelpcdn.com/bphoto/Qmu73I...,False,https://www.yelp.com/biz/dog-haus-eastvale-2?a...,149,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 33.97545, 'longitude': -117.58982}","[pickup, delivery]",$$,"{'address1': '14063 Limonite Ave', 'address2':...",+19099233000,(909) 923-3000,18086.549684
38,3r4wbjqpkOliDnMpSQXb5w,urban-dripp-riverside-5,URBAN DRIPP,https://s3-media2.fl.yelpcdn.com/bphoto/B3BXRg...,False,https://www.yelp.com/biz/urban-dripp-riverside...,337,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",4.5,"{'latitude': 33.98158470860344, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '3750 University Ave', 'address2'...",+19517425949,(951) 742-5949,4542.659417
39,39GiPHLWYISQCu60wkD2yw,stone-church-brewing-corona,Stone Church Brewing,https://s3-media2.fl.yelpcdn.com/bphoto/iMh7yP...,False,https://www.yelp.com/biz/stone-church-brewing-...,173,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 33.812248, 'longitude': -117.507637}",[],$$,"{'address1': '2785 Cabot Dr', 'address2': 'Ste...",+19514330121,(951) 433-0121,17887.632727
